In [ ]:
import pandas as pd
from random import choice
import numpy as np

In [ ]:
train_data = pd.read_csv('/kaggle/input/house-price/train.csv')

In [ ]:
y=train_data.pop('Sold Price')
train_data=train_data.drop(columns=['Id'])

In [ ]:
train_data

In [ ]:
test_data = pd.read_csv('/kaggle/input/house-price/test.csv')

In [ ]:
test_sub=test_data.pop('Id')

In [ ]:
test_data

In [ ]:
data=pd.concat([train_data,test_data])

In [ ]:
data.drop(columns=['Address','Summary','Elementary School','Middle School','High School','Heating features','Cooling features','City'],inplace=True)

In [ ]:
data

In [ ]:
list(data)

In [ ]:
#pd.set_option("display.max_rows", None)

In [ ]:
def fillna_most(data,name):
    data[name].fillna(data[name].value_counts().iloc[0],inplace=True)

In [ ]:
def devide(data,name):
    data[name]=data[name].fillna('unknown')
    data[name]=data[name].map(lambda x:x.split(', '))
    all=[]
    for i in data[name]:
        for j in i:
            if j not in all:
                all.append(j)
    print("'%s' gets %d new features"%(name,len(all)))
    all.sort()
    for i in all:
        data[name+'_'+str(i)]=data[name].map(lambda x:1 if i in x else 0)
    data.drop(columns=[name],inplace=True)

In [ ]:
def devide_time(data,name):
    data[name]=data[name].map(lambda x:x.split('-'))
    data[name+'_year']=data[name].map(lambda x:int(x[0]))
    data[name+'_month']=data[name].map(lambda x:int(x[1]))
    data[name+'_day']=data[name].map(lambda x:int(x[2]))
    data.drop(columns=[name],inplace=True)

In [ ]:
def one_hot(data,name):
    temp=list(data[name].unique())
    for i in temp:
        data[name+'_'+str(i)]=data[name].map(lambda x:1 if x==i else 0)
    data.drop(columns=[name],inplace=True)

In [ ]:
devide(data,'Type')
devide(data,'Heating')
devide(data,'Cooling')
devide(data,'Parking')
devide(data,'Bedrooms')
devide(data,'Flooring')
devide(data,'Appliances included')
devide(data,'Laundry features')
devide(data,'Parking features')

In [ ]:
devide_time(data,'Listed On')
data['Last Sold On'].fillna('0-0-0',inplace=True)
devide_time(data,'Last Sold On')

In [ ]:
fillna_most(data,'Lot')
fillna_most(data,'Bathrooms')
fillna_most(data,'Full bathrooms')
fillna_most(data,'Total interior livable area')
fillna_most(data,'Total spaces')
fillna_most(data,'Garage spaces')
fillna_most(data,'Region')
fillna_most(data,'Elementary School Score')
fillna_most(data,'Elementary School Distance')
fillna_most(data,'Middle School Score')
fillna_most(data,'Middle School Distance')
fillna_most(data,'High School Score')
fillna_most(data,'High School Distance')
fillna_most(data,'Tax assessed value')
fillna_most(data,'Annual tax amount')

In [ ]:
one_hot(data,'Region')
one_hot(data,'State')

In [ ]:
temp=list(data['Year built'].unique())
years=[]
for i in temp:
    if i==i and i!=0.0 and i !=19.0 and i!=80.0 and i!=2281.0 and i!=9999.0 and i!=1006.0 and i!=1471.0:
        years.append(i)
data['Year built']=data['Year built'].map(lambda x:choice(years) if x in [0.0,19.0,80.0,2281.0,9999.0] else x)
data['Year built'].fillna(choice(years),inplace=True)

In [ ]:
data['Garage spaces']=data['Garage spaces'].map(lambda x:data['Garage spaces'].value_counts().iloc[0] if x<0 else x)

In [ ]:
data['Listed Price']=data['Listed Price'].map(lambda x:data['Listed Price'].value_counts().iloc[0] if x==0 else x)

In [ ]:
data['Last Sold Price'].fillna(0,inplace=True)

In [ ]:
data['Zip_National_Area']=data['Zip'].map(lambda x:int(str(x)[0]))
data['Zip_Sectional_Center']=data['Zip'].map(lambda x:int(str(x)[1:3]))
data['Zip_Delivery_Area']=data['Zip'].map(lambda x:int(str(x)[3:]))
data.drop(columns=['Zip'],inplace=True)

In [ ]:
data.columns[np.where(np.isnan(data))[1]]

In [ ]:
test_data=data[47439:]
train_data=data[:47439]

In [ ]:
train_data.to_csv('train_modified.csv',index=False)
test_data.to_csv('test_modified.csv',index=False)

In [ ]:
train_data

In [ ]:
test_data

In [ ]:
X=train_data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.2, random_state = 1)

In [ ]:
# from sklearn.preprocessing import StandardScaler
# ss_x = StandardScaler()
# X_train = ss_x.fit_transform(X_train)
# X_test = ss_x.transform(X_test)
# ss_y = StandardScaler()
# y_train = ss_y.fit_transform(y_train.values.reshape(-1, 1))
# y_test = ss_y.transform(y_test.values.reshape(-1, 1))
# test_data = ss_x.transform(test_data)

In [ ]:
from sklearn import ensemble
clf = ensemble.GradientBoostingRegressor(verbose=True)
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

In [ ]:
# from sklearn.svm import SVR
# svr = SVR(kernel='poly',verbose=True) 
# svr.fit(X_train, y_train)
# svr.score(X_test,y_test)

In [ ]:
# from sklearn.tree import DecisionTreeRegressor
# tree_reg=DecisionTreeRegressor(max_depth=2)
# tree_reg.fit(X_train, y_train)
# tree_reg.score(X_test,y_test)

In [ ]:
# from sklearn.linear_model import LinearRegression
# LR = LinearRegression()
# LR.fit(X_train, y_train)
# LR.score(X_test,y_test)

In [ ]:
y_pred = clf.predict(test_data)

In [ ]:
import joblib
joblib.dump(clf, 'model.pkl')
# clf = joblib.load('model.pkl')

In [ ]:
price = []
for i in range(len(test_sub)):
    price.append(y_pred[i])
df_price = pd.DataFrame(price)
# 将id列和预测出来的房价列拼接并输出submission文件
df=pd.concat([test_sub,df_price],axis=1)
df.to_csv('submission.csv',header=['Id','Price'], index=False)

In [ ]:
sub = pd.read_csv('submission.csv')

In [ ]:
sub